In [ ]:
import numpy as np
import re
import os
import h5py
from sklearn.preprocessing import LabelBinarizer
import keras
from keras import optimizers
from keras.callbacks import TensorBoard
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, GlobalAveragePooling2D, add
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization, ZeroPadding2D
from keras import regularizers
from keras import backend as K
from VGG19_utils import *
from VGG19_preprocess import *

In [ ]:
def VGG19(input_shape, num_classes, dropout, weight_decay):
    model = Sequential()
    ######################################## STAGE 1 ####################################################################
    model.add(Conv2D(64, (3,3), padding = 'same', kernel_regularizer = regularizers.l2(weight_decay), name = '1_1', 
                    input_shape = input_shape))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.3))
    
    model.add(Conv2D(64, (3,3), padding = 'same', kernel_regularizer = regularizers.l2(weight_decay), name = '1_2'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    
    model.add(MaxPooling2D(pool_size=  (2,2), name = '1_pool'))
    
    ######################################## STAGE 2 ####################################################################
    model.add(Conv2D(128, (3,3), padding = 'same', kernel_regularizer = regularizers.l2(weight_decay), name = '2_1'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.4))
    
    model.add(Conv2D(128, (3,3), padding = 'same', kernel_regularizer = regularizers.l2(weight_decay), name = '2_2'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    
    model.add(MaxPooling2D(pool_size= (2,2), name = '2_pool'))
    
    ######################################## STAGE 3 ####################################################################
    model.add(Conv2D(256, (3,3), padding = 'same', kernel_regularizer = regularizers.l2(weight_decay), name = '3_1'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.4))
    
    model.add(Conv2D(256, (3,3), padding = 'same', kernel_regularizer = regularizers.l2(weight_decay), name = '3_2'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.4))
    
    model.add(Conv2D(256, (3,3), padding = 'same', kernel_regularizer = regularizers.l2(weight_decay), name = '3_3'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    
    model.add(MaxPooling2D(pool_size= (2,2), name = '3_pool'))
    
    ######################################## STAGE 4 ####################################################################
    model.add(Conv2D(512, (3,3), padding = 'same', kernel_regularizer = regularizers.l2(weight_decay), name = '4_1'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.4))
    
    model.add(Conv2D(512, (3,3), padding = 'same', kernel_regularizer = regularizers.l2(weight_decay), name = '4_2'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.4))
    
    model.add(Conv2D(512, (3,3), padding = 'same', kernel_regularizer = regularizers.l2(weight_decay), name = '4_3'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    
    model.add(MaxPooling2D(pool_size= (2,2), name = '4_pool'))
    
    ######################################## STAGE 5 ####################################################################
    model.add(Conv2D(512, (3,3), padding = 'same', kernel_regularizer = regularizers.l2(weight_decay), name = '5_1'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.4))
    
    model.add(Conv2D(512, (3,3), padding = 'same', kernel_regularizer = regularizers.l2(weight_decay), name = '5_2'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(0.4))
    
    model.add(Conv2D(512, (3,3), padding = 'same', kernel_regularizer = regularizers.l2(weight_decay), name = '5_3'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    
    model.add(MaxPooling2D(pool_size= (2,2), name = '5_pool'))
    model.add(Dropout(0.5))
    
    ######################################## Modification for Caltech 101 ################################################
    model.add(Flatten(name='flatten'))
    model.add(Dense(4096, use_bias = True, name='fc_caltech101'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(dropout))
    model.add(Dense(4096, name='fc2'))  
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(dropout))      
    model.add(Dense(num_classes, name='predictions_caltech101'))
    model.add(BatchNormalization())
    model.add(Activation('softmax'))
    
    return model
    

In [ ]:
model = VGG19(input_shape=(64,64,3), num_classes = 2, dropout=0.5, weight_decay=0.0001)

In [ ]:
model.compile(optimizer='adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [ ]:
X_train_orig, Y_train_orig, X_test_orig, Y_test_orig = load_dataset()
#Normalise Image vectors
X_train = X_train_orig/255.
X_test  =X_test_orig/255.

#Convert Training and test labels to one hot vectors
Y_train = convert_to_one_hot(Y_train_orig,6).T
Y_test = convert_to_one_hot(Y_test_orig, 6).T

print ("number of training examples = " + str(X_train.shape[0]))
print ("number of test examples = " + str(X_test.shape[0]))
print ("X_train shape: " + str(X_train.shape))
print ("Y_train shape: " + str(Y_train.shape))
print ("X_test shape: " + str(X_test.shape))
print ("Y_test shape: " + str(Y_test.shape))

In [ ]:
history = model.fit(X_train, Y_train, epochs = 50, batch_size = 32)

In [ ]:
preds = model.evaluate(X_test, Y_test)
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))

In [ ]:
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('Model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()